## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2021-10-11 11:38:03--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 103.252.115.157, 2001::6729:840c
Connecting to www.dropbox.com (www.dropbox.com)|103.252.115.157|:443... failed: Connection timed out.
Connecting to www.dropbox.com (www.dropbox.com)|2001::6729:840c|:443... failed: Network is unreachable.


In [2]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [7]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(d.lower()) for d in data]

In [8]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [9]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [11]:
?Word2Vec

In [12]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [13]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.42607   , -1.0877861 ,  0.15885843,  3.9365609 ,  1.5686439 ,
        1.3799038 ,  2.9481127 , -2.0994647 ,  0.16529751,  3.2495859 ,
       -0.35359487,  1.8322489 ,  4.6274533 ,  0.8774054 ,  3.2461774 ,
        0.1057987 ,  0.39118424, -0.4846263 ,  0.5536827 , -3.4911332 ,
       -3.549663  , -1.415887  , -0.58423185, -0.45246896,  0.80589885,
       -2.3611104 , -0.24042457, -1.2606723 ,  0.08598353,  1.7714092 ,
       -0.7822217 , -0.32408914], dtype=float32)

In [14]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9492585062980652),
 ('sauce', 0.9262681603431702),
 ('butter', 0.9254346489906311),
 ('pasta', 0.9170589447021484),
 ('corn', 0.914945662021637),
 ('beans', 0.9129352569580078),
 ('cheese', 0.9109046459197998),
 ('chocolate', 0.9084083437919617),
 ('fruit', 0.9057673811912537),
 ('honey', 0.9038717150688171)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [15]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [16]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.47964534163475037)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [23]:
model.index_to_key

['<user>',
 '.',
 ':',
 'rt',
 ',',
 '<repeat>',
 '<hashtag>',
 '<number>',
 '<url>',
 '!',
 'i',
 'a',
 '"',
 'the',
 '?',
 'you',
 'to',
 '(',
 '<allcaps>',
 '<elong>',
 ')',
 'me',
 'de',
 '<smile>',
 '！',
 'que',
 'and',
 '。',
 '-',
 'my',
 'no',
 '、',
 'is',
 'it',
 '…',
 'in',
 'n',
 'for',
 '/',
 'of',
 'la',
 "'s",
 '*',
 'do',
 "n't",
 'that',
 'on',
 'y',
 "'",
 'e',
 'o',
 'u',
 'en',
 'this',
 'el',
 'so',
 'be',
 "'m",
 'with',
 'just',
 '>',
 'your',
 '^',
 'like',
 'have',
 'te',
 'at',
 '？',
 'love',
 'se',
 'are',
 '<',
 'm',
 'r',
 'if',
 'all',
 'b',
 '・',
 'not',
 'but',
 'we',
 'es',
 'ya',
 '&',
 'follow',
 'up',
 'what',
 'get',
 'lol',
 'un',
 '♥',
 'lo',
 'when',
 'was',
 '“',
 '”',
 'one',
 'por',
 'si',
 'out',
 '_',
 'mi',
 'can',
 '<sadface>',
 'من',
 '♡',
 '´',
 'he',
 'con',
 'they',
 'now',
 'go',
 '،',
 'para',
 'los',
 'know',
 'haha',
 'good',
 'tu',
 'back',
 '~',
 'about',
 'new',
 ';',
 'as',
 'day',
 'how',
 'who',
 'will',
 'want',
 'people',
 'y

In [22]:
# words = sorted(model.key_to_index.keys(), 
#                key=lambda word: model.vocab[word].count,
#                reverse=True)[:1000]
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [27]:
# for each word, compute it's vector with model
word_vectors = [model.get_vector(w) for w in words]
word_vectors = np.asarray(word_vectors)

In [28]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [42]:
print(word_vectors.shape)

(1000, 100)


In [51]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance

word_vectors_pca = PCA(n_components=2)
word_vectors_pca = word_vectors_pca.fit_transform(word_vectors_)
print(word_vectors_pca.shape)

# and maybe MORE OF YOUR CODE here :)
from sklearn.preprocessing import StandardScaler
word_vectors_pca = StandardScaler().fit_transform(word_vectors_pca)

(1000, 2)


In [52]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [53]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [54]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [55]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
word_tsne = StandardScaler().fit_transform(word_tsne)

In [56]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1110', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [60]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    tok_vectors = [model.get_vector(tok) for tok in tokenizer.tokenize(phrase.lower()) if tok in model.key_to_index]
    if len(tok_vectors) == 0:
        return vector
    vector = np.mean(tok_vectors, axis=0)
    return vector
        
    

In [61]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [65]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.asarray([get_phrase_embedding(p) for p in chosen_phrases])


In [66]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [67]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [68]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1226', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [69]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [124]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_emb = get_phrase_embedding(query)
#     print(data_vectors[0], data_vectors[0].shape, query_emb.shape)
#     print(np.cos(query_emb, data_vectors[0]))
    query_sim = [1 - np.dot(query_emb, dv) / (np.linalg.norm(query_emb) * np.linalg.norm(dv)) for dv in data_vectors]
    topk_sim_idxs = [i for i in np.argpartition(query_sim, k)][:k+10]
    print(topk_sim_idxs)
    print('\n'.join(['%s\t%f' % (data[idx], query_sim[idx]) for idx in topk_sim_idxs]))
    nearests = sorted([(data[idx], query_sim[idx]) for idx in topk_sim_idxs], key=lambda x: x[-1])
    nearests = [n[0] for n in nearests]
    
    return nearests


In [125]:
results = find_nearest(query="How do i enter the matrix?", k=10)

[358183, 358184, 358178, 358179, 358180, 358181, 358182, 358176, 358187, 358188, 358189, 358175, 358185, 358186, 358177, 358191, 358193, 358174, 358173, 358172]
How are molarity and density related?
	0.983731
Why are Samsung phones catching fire?
	0.983731
Will an exact simulation of my brain create a robot which behaves exactly like me where intelligence and consciousness is concerned? Why?
	0.983731
In International ODI/T20 cricket matches, if a batsman hits a ball which is above the head height (and which is not a full-toss), is it a no-ball? ?
	0.983731
Which are some of the low cost services for startups?
	0.983731
Why didn't India recapture Pak-occupied Kashmir during the war of 1971 when India won?
	0.983731
What are the major features of Chhattisgarh geography?
	0.983731
Which private medical college should I apply when my NEET score is 184?
	0.983731
What do you do if you hate your child?
	0.983731
When is next iPhone release in 2016?
	0.983731
I have two passions, programming

In [113]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

[358183, 358184, 358178, 358179, 358180, 358181, 358182, 358176, 358187, 358188]
How are molarity and density related?
Why are Samsung phones catching fire?
Will an exact simulation of my brain create a robot which behaves exactly like me where intelligence and consciousness is concerned? Why?
In International ODI/T20 cricket matches, if a batsman hits a ball which is above the head height (and which is not a full-toss), is it a no-ball? ?
Which are some of the low cost services for startups?
Why didn't India recapture Pak-occupied Kashmir during the war of 1971 when India won?
What are the major features of Chhattisgarh geography?
Which private medical college should I apply when my NEET score is 184?
What do you do if you hate your child?
When is next iPhone release in 2016?



AssertionError: 

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.